In [32]:
import tensorflow as tf

sh = [5,2,2,1]
x = tf.ones(sh)
y = tf.ones(sh)
n = x + y
k = x + y + y
ls = [x,n,k,k]

print (mod_ls[0].get_shape().as_list())
z = tf.reduce_mean(ls, 0)
p = k.get_shape().as_list()


res = tf.concat(ls, axis=3)
mx = tf.reduce_max(res)
print (res.get_shape().as_list())
with tf.Session() as sess:
    print (sess.run(res))
    print (sess.run(mx))

[5, 2, 2]
[5, 2, 2, 4]
[[[[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]

  [[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]]


 [[[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]

  [[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]]


 [[[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]

  [[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]]


 [[[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]

  [[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]]


 [[[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]

  [[ 1.  2.  3.  3.]
   [ 1.  2.  3.  3.]]]]
3.0


In [33]:
for i in range(0, 10):
    print (i)

0
1
2
3
4
5
6
7
8
9


In [71]:
import numpy as np

# Following method creates the pixel wise mode tensor.
# Expects input of the type [batch, height, width, num_MC_trials]
def get_mode_tensor(input):
    sh = input.get_shape().as_list()
    batch = sh[0]
    height = sh[1]
    width = sh[2]
    num_mc_trials = sh[3]
    print (batch, height, width, num_mc_trials)
    #res_arr = np.zeros([batch, height, width, 1])
    res_arr = get_multidimensional_list(batch, height, width)
    for i in range(0, batch):
        # reduced_tensor has dimension [height, width, num_MC_trials]
        reduced_tensor = input[i]
        uni_dim_tensor = []
        for x in range(0, height):
            for y in range(0, width):
                uni_dim_tensor = create_uni_dim_tensor(reduced_tensor, x, y)
                y_val, y_indx, y_count = tf.unique_with_counts(uni_dim_tensor)
                
                #TODO: Change this to (1 - fx/S)
                res_arr[i][x][y][0] = 1 - (tf.reduce_max(y_count)/num_mc_trials)
    return tf.convert_to_tensor(res_arr)
        
    
# Following method expects input of the form [height, width, channels]
def create_uni_dim_tensor(input, height_ind, width_ind):
    res = []
    num_channels = input.get_shape().as_list()[2]
    for i in range(0, num_channels):
        res.append(input[height_ind][width_ind][i])
    return tf.convert_to_tensor(res)


def get_multidimensional_list(batch, height, width):
    ls = [[[[None]*1]*width]*height]*batch
    return ls

mode_tensor = get_mode_tensor(res)
print (mode_tensor.get_shape().as_list())

5 2 2 4
[5, 2, 2, 1]


0.0


None
